In [2]:
# export openblas_num_threads=1; export OMP_NUM_THREADS=1; python simu.py

from utils import * 
from weight import * 
from simu_funs import *  
from main_200 import *
a = now()

rep_times = 96
region_parallel = False
full_parallel = False
pattern_seed = 2
sd_u_O = 25
w_O = .5
w_A = 1.5
u_D = 80
thre_range = [100, 101, 105, 110]

penalty_range = [[3e-4, 1e-4, 5e-5], [3e-4, 1e-4, 5e-5]]
with_NO_MARL = True
with_IS = True
with_MF = True
n_layer = 3
max_iteration = 5001
Learning_rate = 5e-4
w_hidden = 30
batch_size = 32


aim = "final_sd"

sd_R = 0
day = 7
results, res_real = [], []


l = 5

npseed(pattern_seed)
u_O = rnorm(100, sd_u_O, l**2) 


print("max(u_O) = ", np.round(max(u_O), 1), "mean(u_O) = ", np.round(np.mean(u_O), 1))

# generate the corresponding target plicy
target_policys = []
n_tp = len(thre_range)
for i in range(n_tp):
    O_thre = thre_range[i]
    print("O_threshold = " + str(thre_range[i]))
    if i == 0: 
        target_policy = simu_target_policy_pattern(l = l, u_O = u_O, threshold =  O_thre, print_flag = "all") # "all"
    else:
        target_policy = simu_target_policy_pattern(l = l, u_O = u_O, threshold =  O_thre, print_flag = "None") # "policy_only"
    target_policys.append(target_policy)

# generate the adj for the grid
neigh = adj2neigh(getAdjGrid(l))


seed = 1 
l = 5
T = day * 48
t_func = None
sd_D = None
sd_O = None
inner_parallel = False
CV_QV = True
penalty_NMF = [[1e-3], [1e-3]]
dim_S_plus_Ts = 3 + 3
epsilon = 1e-6

##########################################################################################################################################################

"""
Output: a len-n-target of len-est results
"""
npseed(seed)
N = l ** 2
def behav(s, a):
    return 0.5
behav = list(itertools.repeat(behav, N))

def Ts(S_neigh):
    return np.mean(S_neigh, 0)
def Ta(A_neigh):
    return Ta_disc(np.mean(A_neigh, 0))

# observed data following behav
data, adj_mat, details = DG_once(seed = seed, l = l, T = T, u_D = u_D, 
                                 u_O = u_O, 
                                 t_func = t_func,  
                                 sd_D = sd_D, sd_R = sd_R, sd_O = sd_O, 
                                 w_A = w_A, w_O = w_O)

target_policy = target_policys[0]

val_paras = [sd_D, sd_R, u_D, sd_O, w_A, w_O, u_O]

# OPE
a = now()
value_targets = []
count = 0
n_target = len(target_policys)
value_estimators = V_DR(data = data, tp = target_policy, bp = behav, 
                        adj_mat = adj_mat, dim_S_plus_Ts = dim_S_plus_Ts, 
                     t_func = t_func, 
                        is_weight_val = True, val_paras = val_paras, 
                     Ts = Ts, Ta = Ta, penalty = penalty_range, penalty_NMF = penalty_NMF, 
                        n_layer = n_layer, CV_QV = CV_QV, 
                    w_hidden = w_hidden, lr = Learning_rate,  
                    batch_size = batch_size, max_iteration = max_iteration, epsilon = epsilon, 
                        with_MF = with_MF, with_NO_MARL = with_NO_MARL, with_IS = with_IS, 
                     inner_parallel = inner_parallel)

max(u_O) =  157.3 mean(u_O) =  93.7
O_threshold = 100
means of Order: 

89.6 98.6 46.6 141.0 55.2 

79.0 112.6 68.9 73.6 77.3 

113.8 157.3 101.0 72.1 113.5 

85.1 99.5 129.4 81.3 100.2 

78.0 96.1 106.4 75.3 91.5 

target policy: 

0 0 0 1 0 

0 1 0 0 0 

1 1 1 0 1 

0 0 1 0 1 

0 0 1 0 0 

number of reward locations:  9
O_threshold = 101
number of reward locations:  8
O_threshold = 105
number of reward locations:  7
O_threshold = 110
number of reward locations:  6
here 200
0 DONE!
2 DONE!
1 DONE!
< ----- den func est in MC: DONE!  -----> 
std among MC reps: 0.0001850215785129844 0.030182201927122277
WEIGHT estimation error (std_MC, std_est, median_diff, R2): 
 [ 3.000e-02  6.400e-01  3.000e-01 -4.412e+02] 

std among MC reps: 0.0002523745133255857 0.03206810377027042
WEIGHT estimation error (std_MC, std_est, median_diff, R2): 
 [ 3.0000e-02  6.6000e-01  4.4000e-01 -4.1323e+02] 

std among MC reps: 0.00023580476413856814 0.035528176151526904
WEIGHT estimation error (std_MC, std_est, m

KeyboardInterrupt: 

In [1]:
from _utility import * 
from weight import * 
from simu_funs import *  
from main import *

In [ ]:
## Sample input for specifying t_func and tp:

### 1. tp depending on order and driver

# assume s = [o, d, mismatch]r
pi_adaptive = []
threshold = 1
for i in range(9):
    def tp_i(s, a = 0, random_choose = False):
        is_reward = int(s[0] / s[1] > threshold) # when order > driver, we give subsidy
        if random_choose:
            return is_reward
        else:
            return int(a == is_reward)
    pi_adaptive.append(tp_i)
target_policy = pi_adaptive

t_func = None

### 2. tp depending on location and time_of_day - option 1 
# (RECOMMENDED) it may reduce variance with such a discretized time stamp

# if you need a more complex dependecy on time and location, please modify this exmaple or let me know

def t_func(t):
    # the time state variable is an indicator for rush hours
    return int(t % 48 in [36,37,38,39,40])
range_transformed_time_index = [0, 1] # can be extended to more than two time slices

location_give_subsidy = [0, 1, 2] # other regions will never receive subsidy

# adaptive_tp_dictionary is a len-N of len-N_range_transformed_time_index of binary value indicating 
# whether or not this space-time slice should receive subsidy
# For example: 

adaptive_action_list = []
adaptive_tp_dictionary = []
for region_index in range(9):
    region_action=[]
    for time_index in range_transformed_time_index:
        if(time_index == 1 and region_index in location_give_subsidy):
            adaptive_action_list.append(1)
            region_action.append(1)
        else:
            adaptive_action_list.append(0)
            region_action.append(0)
    adaptive_tp_dictionary.append(region_action)
print(adaptive_tp_dictionary)


pi_adaptive = []
for i in range(9):
    def tp_i(s, a = 0, random_choose = False, fixed_policy_i = adaptive_tp_dictionary[i]): # , fixed_policy_i = adaptive_tp_dictionary[i]
        t = int(s[3]) # already transformed by t_func
        if random_choose:
            return fixed_policy_i[t]
        else:
            return int(a == fixed_policy_i[t])
    pi_adaptive.append(tp_i)
target_policy = pi_adaptive


### 3. tp depending on location and time_of_day - option 2
def t_func(t):
    return int(t % 48)

adaptive_action_list = []
adaptive_tp_dictionary = []
for region_index in range(9):
    region_action=[]
    for time_index in range(336):
        if(time_index%48 in [36,37,38,39,40]):
            adaptive_action_list.append(1)
            region_action.append(1)
        else:
            adaptive_action_list.append(0)
            region_action.append(0)
    adaptive_tp_dictionary.append(region_action)
# print(adaptive_tp_dictionary)

pi_adaptive = []
for i in range(9):
    def tp_i(s, a = 0, random_choose = False, fixed_policy_i = adaptive_tp_dictionary[i]):
        t = int(s[3])
        if random_choose:
            return fixed_policy_i[t]
        else:
            return int(a == fixed_policy_i[t])
    pi_adaptive.append(tp_i)
target_policy = pi_adaptive


### 4. tp depending on location, time_of_day and day_of_week

day_of_week_need_subsidy = [0,1,2,3,4] # for example, only subsidy for rush hours in weekdays
time_index_range_in_a_week_need_subsidy = []
for day in day_of_week_need_subsidy:
    time_index_range_in_a_week_need_subsidy += list(arr([36,37,38,39,40]) * day)


def t_func(t):
    # the time state variable is an indicator for hours in week that need subsidy
    return int(t % (48 * 7) in time_index_range_in_a_week_need_subsidy)
range_transformed_time_index = [0, 1] # can be extended to more than two time slices

location_give_subsidy = [0, 1, 2] # other regions will never receive subsidy

# adaptive_tp_dictionary is a len-N of len-N_range_transformed_time_index of binary value indicating 
# whether or not this space-time slice should receive subsidy
# For example: 

adaptive_action_list = []
adaptive_tp_dictionary = []
for region_index in range(9):
    region_action=[]
    for time_index in range_transformed_time_index:
        if(time_index == 1 and region_index in location_give_subsidy):
            adaptive_action_list.append(1)
            region_action.append(1)
        else:
            adaptive_action_list.append(0)
            region_action.append(0)
    adaptive_tp_dictionary.append(region_action)

print(adaptive_tp_dictionary)


pi_adaptive = []
for i in range(9):
    def tp_i(s, a = 0, random_choose = False, fixed_policy_i = adaptive_tp_dictionary[i]): # , fixed_policy_i = adaptive_tp_dictionary[i]
        t = int(s[3]) # already transformed by t_func
        if random_choose:
            return fixed_policy_i[t]
        else:
            return int(a == fixed_policy_i[t])
    pi_adaptive.append(tp_i)
target_policy = pi_adaptive